In [1]:
from whoisthat.whoisthat import get_summary, spoiler_check, get_summary_from_text

In [2]:
# import yaml
# with open('whoisthat/database.yml', 'r') as file:
#     print(yaml.safe_load(file))

# Pride and Prejucice
1. Place a bookmark in your book
2. Generate a summary of the character of interest
3. Check for spoilers

In [3]:
book = 'Pride and Prejudice'
bookmark = 'Volume I' # e.g. Volume or Chapter
character = 'Mr. Wickham'
pride_and_prejudice_summary = get_summary(book, bookmark, character)
print(pride_and_prejudice_summary)

Mr. Wickham: charmed Bingley, deceived Lydia Bennet with false promises, and fled to avoid marriage consequences.


In [4]:
pride_and_prejudice_spolier_check = spoiler_check(book, character, pride_and_prejudice_summary)
print(pride_and_prejudice_spolier_check)

Yes.

The summary mentions Lydia Bennet being deceived by Mr. Wickham's false promises, which implies that he betrayed her trust and ultimately eloped with her, matching the spoiler you provided.


# A Game of Thrones

In [5]:
book = 'A Game of Thrones'
bookmark = 'Chapter 1'
character = 'Jon Snow'
game_of_thrones_summary = get_summary(book, bookmark, character)
print(game_of_thrones_summary)

Jon Snow has arrived at Winterfell, met his aunt Lady Catelyn Stark, and been appointed to the Night's Watch.


In [6]:
print(spoiler_check(book, character, game_of_thrones_summary))

No.

This summary does not include any information about Jon Snow's parentage or his claim to the Iron Throne, so the spoiler is not present in this text.


In [7]:
tyrion_summary = get_summary(book, bookmark, 'Tyrion Lannister')
print(spoiler_check(book, 'Tyrion Lannister', tyrion_summary))

No spoilers in the database for Tyrion Lannister in A Game of Thrones.


# Harry Potter

In [8]:
book = 'Harry Potter and the Half-Blood Prince'
character = 'Albus Dumbledore'

chapter_20_summary = get_summary(book, 'Chapter 20', character)
print("Chapter 20 summary:")
print(chapter_20_summary)
print("")
print("")
print("Spoiler check for Chapter 20:")
print(spoiler_check(book, character, chapter_20_summary))
print("")
print("")

chapter_29_summary = get_summary(book, 'Chapter 29', character)
print("Chapter 29 summary:")
print(chapter_29_summary)
print("")
print("")
print("Spoiler check for Chapter 29:")
print(spoiler_check(book, character, chapter_29_summary))

Chapter 20 summary:
Here is a summary of Dumbledore's actions up to Chapter 20:

Dumbledore reveals his past, mentors Harry, and sacrifices himself.


Spoiler check for Chapter 20:
Yes.

The reason is because the summary explicitly mentions "sacrifices himself", which implies that Dumbledore's death is intentional, and it matches the spoiler "Albus Dumbledore is killed by Severus Snape".


Chapter 29 summary:
Dumbledore revealed Hogwarts' secrets to Harry, protected him from Death Eaters, and manipulated Snape's loyalties.


Spoiler check for Chapter 29:
Yes.

The reason I answered "yes" is because the summary states that Dumbledore manipulated Snape's loyalties, which implies that Dumbledore had some control over Snape, but ultimately, Snape kills Dumbledore in 'Harry Potter and the Half-Blood Prince', contradicting Dumbledore's supposed manipulation of Snape.


# Reinforcing REG

1. Load the book what we wrote with AI and choose a bookmark
2. Generate a summary
3. Check for spoilers

In [3]:
with open('../data/ReinforcingREG.txt', 'r') as file:
  text = file.read()
bookmark = 'Chapter 1'
character = 'Reginald'
reg_story_ch1_summary = get_summary_from_text(text, bookmark, character)
print(reg_story_ch1_summary)

Reginald has reprimanded Sylvia, then forgiven her after learning about Draco's blackmail threats.
